In [88]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats as stats
import io

### Coverage loss

In [89]:
# Let's start with the insurance loss data (extracted data from CBPP table 1
coverage_loss = pd.read_csv('data/Medicaid Work Requirements Table.csv')

coverage_loss.set_index('State',inplace=True)

In [90]:
# Form of data I need:
index = coverage_loss.index[1:]
# P:pop 19--64, p:pop in ins data, i:insured, pm &im: margin of errors to p and i, 'il':insurance loss & 'dr' is death rates
columns = ['P','p','i','pm','im','il','dr'] 
df = pd.DataFrame(index=index,columns=columns)

### Population & Current insurance rates
For national level estimates, We use data from [National Population Projection Datasets](https://www.census.gov/data/datasets/2023/demo/popproj/2023-popproj.html) to inform 19--64 age population from 2025. Same dataset doesn't have state-level data. So, I am going to use ACS data that has 2023 data for both state-level insurance coverage and population. 

**to do:** convert 2023 population numbers to match national 2025 number in a reasonable way.

In [91]:
# read insurance data from US Census Bureau (https://data.census.gov/table/ACSST5Y2023.S2701?q=health%20insurance) 
# 2023: ACS 5-year estimates subject table
ins_ = pd.read_excel('data/State-level-Insurance-numbers.xlsx',sheet_name='Data',header=[0,1,2],index_col=0)


In [92]:
# all the states
ins_.columns.get_level_values(0).unique().tolist()

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',
 'Puerto Rico']

#### ensuring state names are same across: 
Difference between cbpp and insurance data in terms of state names so far only seems to be that cbpp has a total row (for all of US), whereas insurance data has Peurto Rico that's not present in cbpp data. 

However, name of Hawaii is Hawai'i in cbpp, so need to correct it, and also need it correct for df. 

In [93]:
coverage_loss.index
coverage_loss.rename(index={'Hawai‘i':'Hawaii'},inplace=True)

df.index
df.rename(index={'Hawai‘i':'Hawaii'},inplace=True)

In [94]:
#of interest:
ins_.index

#age-groups of interest:
ages = ['19 to 25 years', '26 to 34 years', '35 to 44 years','45 to 54 years', '55 to 64 years'] 
#ins.loc[ages,[('Alabama', 'Total', 'Estimate'), ('Alabama', 'Insured', 'Estimate')]]

#Insurance rates by age
#Have to convert columns to numeric by removing commas first.¶
ins_.loc[ages] = ins_.loc[ages].replace({',': '', '±': ''}, regex=True)
ins_.loc[ages] = ins_.loc[ages].apply(pd.to_numeric, errors='coerce')



ins_.loc[ages]

Alabama                                           \
                  Total                  Insured                   
Label          Estimate Margin of Error Estimate Margin of Error   
19 to 25 years   464421            3054   382916            3581   
26 to 34 years   570543            2759   466164            4107   
35 to 44 years   607461            1617   511731            3456   
45 to 54 years   610595            1597   531203            2891   
55 to 64 years   657985             884   598075            2278   

                                                                          \
               Percent Insured                 Uninsured                   
Label                 Estimate Margin of Error  Estimate Margin of Error   
19 to 25 years             NaN             0.6     81505            2804   
26 to 34 years             NaN             0.6    104379            3361   
35 to 44 years             NaN             0.5     95730            3273   
45 to 54 years             NaN             0.4     79392            2403   
55 to 64 years             NaN             0.3     59910            2059   

                                                  ... Puerto Rico  \
               Percent Uninsured                  ...       Total   
Label                   Estimate Margin of Error  ...    Estimate   
19 to 25 years               NaN             0.6  ...      306062   
26 to 34 years               NaN             0.6  ...      371358   
35 to 44 years               NaN             0.5  ...      383494   
45 to 54 years               NaN             0.4  ...      411933   
55 to 64 years               NaN             0.3  ...      438701   

                                                                         \
                                Insured                 Percent Insured   
Label          Margin of Error Estimate Margin of Error        Estimate   
19 to 25 years            2159   281422            2401             NaN   
26 to 34 years            2050   322283            2712             NaN   
35 to 44 years             710   345822            2138             NaN   
45 to 54 years             687   377423            1895             NaN   
55 to 64 years             291   407190            1484             NaN   

                                                                            \
                               Uninsured                 Percent Uninsured   
Label          Margin of Error  Estimate Margin of Error          Estimate   
19 to 25 years             0.4     24640            1357               NaN   
26 to 34 years             0.6     49075            2215               NaN   
35 to 44 years             0.5     37672            2050               NaN   
45 to 54 years             0.4     34510            1823               NaN   
55 to 64 years             0.3     31511            1430               NaN   

                                
                                
Label          Margin of Error  
19 to 25 years             0.4  
26 to 34 years             0.6  
35 to 44 years             0.5  
45 to 54 years             0.4  
55 to 64 years             0.3  

[5 rows x 520 columns]

In [85]:
ins = ins_.loc[ages].sum()
ins

                                Label          
Alabama      Total              Estimate           2911005
                                Margin of Error       9911
             Insured            Estimate           2490089
                                Margin of Error      16313
             Percent Insured    Estimate                 0
                                                    ...   
Puerto Rico  Percent Insured    Margin of Error        2.2
             Uninsured          Estimate            177408
                                Margin of Error       8875
             Percent Uninsured  Estimate                 0
                                Margin of Error        2.2
Length: 520, dtype: object

In [104]:
## Testing
insured = ins[('Alabama','Insured','Estimate')]
total = ins[('Alabama','Total','Estimate')]
insured_margin = ins[('Alabama','Insured','Margin of Error')]
total_margin = ins[('Alabama','Total','Margin of Error')]
total_margin

9911

In [110]:
for state in df.index:
    # 'p','i','pm','im'
    df.loc[state,'p'] = ins[(state,'Total','Estimate')]
    df.loc[state,'i'] = ins[(state,'Insured','Estimate')]
    df.loc[state,'pm'] = ins[(state,'Total','Margin of Error')]
    df.loc[state,'im'] = ins[(state,'Insured','Margin of Error')]
    df.loc[state,'il'] = 1000*coverage_loss.loc[state,'All adults at risk (thousands)'] #for insurance loss

# for now let's assume 2023 p is same as 2025 P, so
df['P'] = df['p']
df

,P,p,i,pm,im,il,dr
State,,,,,,,
Alabama,2911005,2911005,2490089,9911,16313,231000,NaN
Alaska,421077,421077,358717,5547,6542,117000,NaN
Arizona,4131317,4131317,3531196,9105,17575,1075000,NaN
Arkansas,1721536,1721536,1499318,7947,11553,310000,NaN
California,23613863,23613863,21305919,19995,39544,8156000,NaN
Colorado,3535116,3535116,3166428,9795,15333,542000,NaN
Connecticut,2141207,2141207,1980752,5733,10370,549000,NaN
Delaware,570460,570460,518850,3227,5660,116000,NaN
District of Columbia,445825,445825,427304,3104,4508,159000,NaN



### State-level death rates

In [111]:
df.to_excel('data/data-for-state-analysis.xlsx')